# Atividade 02: Busca heurística com A*

## 1) Parte manual

### 1.1) Formulação

1) **Problema:** Jogo dos 8 números

2) **Estado inicial:** Configuração escolhida do tabuleiro, em um espaço 3 por 3 para o jogo dos 8 números, contendo os números entre 1 a 8, sendo 0 considerado como espaço vazio, disposto de maneira arbitrária. Para a atividade, selecionamos o estado inicial abaixo:


$$
E_i = \Bigg[ \begin{matrix}
    1 & 8 & 2 \\
    & 4 & 3 \\
    7 & 6 & 5 \\
\end{matrix} \Bigg]
$$

3) **Estado final:** Configuração a ser alcançada pelo algoritmo <strong>A*</strong>, sendo ela a ordenação de forma crescente, da esquerda para a direita, de cima para abaixo, sendo a ultima entrada como espaço vazio (sendo na representação o zero). O estado final é dado abaixo:

$$
E_f = \Bigg[ \begin{matrix}
    1 & 2 & 3 \\
    4 & 5 & 6 \\
    7 & 8 &   \\
\end{matrix} \Bigg]
$$

4) **Ações  e custos:** Movimentos unitarios, sendo eles em direções fixas: cima, baixo, esquerda e direita. Todos os movimentos dentro do espaço 3 por 3. Sendo os movimentos unitários, todos tem o custo 1 a cada estado.

5) **Função de avaliação:** Como dada na questão, a função de avaliação do algoritmo <strong>A*</strong> é dada por:

$$
f(n) = g(n) + h(n)
$$

Sendo:

* $g(n)$: Custo do movimento naquele estado, desde o estado inicial, sendo o número de movimentos realizados ate o momento $n$.

* $h(n)$: Função heuristica a ser aplicada, que será definida para o problema como a distância de manhattan (Definido no próximo ponto).


6) **Função heurística:** A função adotada para o problema dos 8 números será a Distância de Manhattan, sendo a mais  comum e mais eficaz para esse tipo de problema. Ela pode ser dada abaixo:

$$
d(\vec{x}, \vec{y}) = \sum_{i = 1}^{n} |x_i - y_i|
$$

* **Descrição:** A distância se basea na soma de distâncias horizontais e verticais de cada peça até a posição de interesse no espaço.

* **Exemplo de uso:**  Dado o numero 1, que esteja na posição espacial na matriz (2, 1), mas sua posição deveria ser (0, 0), a distância de manhattan será expressa por:

$$
d(\vec{x}, \vec{y}) = \sum_{i = 1}^{n} |x_i - y_i| = |2 - 0| + |1 - 0| = 3
$$

### 1.2) Desenho de níveis

<p align="center" >
    <img src="https://raw.githubusercontent.com/Manuelfjr/agentes/refs/heads/develop/assets/atv02-img01.png?token=GHSAT0AAAAAAC6QBQKKCFT6WXB3DVEWKN3U2AFD6WQ" alt="atv02-img01" width="500"/>
</p>


## 2) Parte Automática:

### Imports

In [ ]:
import sys
from nb_utils import set_root, create_dir
PROJECT_DIR = set_root(1)

In [12]:
import heapq
import copy

### Methods

In [13]:
class Tee:
    def __init__(self, file_path):
        self.file = open(file_path, "w")
        self.stdout = sys.stdout  # Salva a saída padrão original

    def write(self, data):
        self.file.write(data)  # Escreve no arquivo
        self.stdout.write(data)  # Escreve no console

    def flush(self):
        self.file.flush()
        self.stdout.flush()

    def close(self):
        self.file.close()
        sys.stdout = self.stdout  # Restaura a saída padrão original

In [14]:
class AStarSolver:
    def __init__(self, initial_state, goal_state, moves, heuristic=None):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.moves = moves
        self.heuristic = heuristic or self.manhattan_distance

    class Node:
        def __init__(self, state, parent=None, move=None, g=0, heuristic_fn=None):
            self.state = state
            self.parent = parent
            self.move = move
            self.g = g
            self.h = heuristic_fn(state) if heuristic_fn else 0
            self.f = self.g + self.h

        def __lt__(self, other):
            return self.f < other.f

    def find_zero(self, state):
        for i, row in enumerate(state):
            if 0 in row:
                return (i, row.index(0))
        return None

    def manhattan_distance(self, state):
        distance = 0
        for i in range(3):
            for j in range(3):
                val = state[i][j]
                if val != 0:
                    goal_i, goal_j = divmod(val - 1, 3)
                    distance += abs(goal_i - i) + abs(goal_j - j)
        return distance

    def is_goal(self, state):
        return state == self.goal_state

    def generate_children(self, state):
        children = []
        x, y = self.find_zero(state)
        for move, (dx, dy) in self.moves.items():
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_state = copy.deepcopy(state)
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                children.append((move, new_state))
        return children

    def print_state(self, state):
        for row in state:
            print(row)
        print()

    def node_name(self, state):
        flat = [str(num) for row in state for num in row]
        return ''.join(flat)

    def reconstruct_path(self, node):
        path = []
        while node:
            path.append((node.move, node.state))
            node = node.parent
        return list(reversed(path))[1:]

    def solve(self):
        start_node = self.Node(self.initial_state, heuristic_fn=self.heuristic)
        frontier = []
        heapq.heappush(frontier, start_node)
        explored = set()
        step = 0

        while frontier:
            print("-" * 30)
            print(f"F{step} =", end=" {")
            print(', '.join([
                f"{self.node_name(n.state)}_{n.f}" for n in frontier
            ]), end="}\n")

            current = heapq.heappop(frontier)

            print("\nNó selecionado (menor f):")
            self.print_state(current.state)
            print(f"f = {current.f} (g = {current.g}, h = {current.h})")

            if self.is_goal(current.state):
                print("✅ Objetivo alcançado!")
                return self.reconstruct_path(current), current.g

            explored.add(str(current.state))
            children = self.generate_children(current.state)

            print("-" * 30)
            print("\nExpandindo nós:")
            for move, child_state in children:
                if str(child_state) not in explored:
                    child_node = self.Node(child_state, parent=current, move=move, g=current.g + 1, heuristic_fn=self.heuristic)
                    heapq.heappush(frontier, child_node)
                    print(f"Movimento: {move}")
                    self.print_state(child_state)
                    print(f"f = {child_node.f} (g = {child_node.g}, h = {child_node.h})")

            print("-" * 30 + "\n")
            step += 1

        print("❌ Nenhuma solução encontrada.")
        return None, None


### Parameters

In [15]:
path_log = PROJECT_DIR / "logs" 
file_path_logger = PROJECT_DIR  / "logs" / "a_star_log.log"

create_dir(path_log)

In [16]:
GOAL_STATE = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

MOVES = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

initial_state = [
    [1, 8, 2],
    [0, 4, 3],
    [7, 6, 5]
]

### Process

In [17]:
tee = Tee(file_path_logger)
sys.stdout = tee

solver = AStarSolver(initial_state, GOAL_STATE, MOVES)
solution_path, total_cost = solver.solve()

if solution_path:
    print("\n--- Caminho da Solução ---")
    for i, (move, state) in enumerate(solution_path):
        print(f"Passo {i+1}: {move}")
        solver.print_state(state)
    print(f"Custo total: {total_cost}")

tee.close()

------------------------------
F0 = {182043765_9}

Nó selecionado (menor f):
[1, 8, 2]
[0, 4, 3]
[7, 6, 5]

f = 9 (g = 0, h = 9)
------------------------------

Expandindo nós:
Movimento: up
[0, 8, 2]
[1, 4, 3]
[7, 6, 5]

f = 11 (g = 1, h = 10)
Movimento: down
[1, 8, 2]
[7, 4, 3]
[0, 6, 5]

f = 11 (g = 1, h = 10)
Movimento: right
[1, 8, 2]
[4, 0, 3]
[7, 6, 5]

f = 9 (g = 1, h = 8)
------------------------------

------------------------------
F1 = {182403765_9, 182743065_11, 082143765_11}

Nó selecionado (menor f):
[1, 8, 2]
[4, 0, 3]
[7, 6, 5]

f = 9 (g = 1, h = 8)
------------------------------

Expandindo nós:
Movimento: up
[1, 0, 2]
[4, 8, 3]
[7, 6, 5]

f = 9 (g = 2, h = 7)
Movimento: down
[1, 8, 2]
[4, 6, 3]
[7, 0, 5]

f = 9 (g = 2, h = 7)
Movimento: right
[1, 8, 2]
[4, 3, 0]
[7, 6, 5]

f = 11 (g = 2, h = 9)
------------------------------

------------------------------
F2 = {102483765_9, 182463705_9, 182743065_11, 082143765_11, 182430765_11}

Nó selecionado (menor f):
[1, 0, 2]
[

## Referencia

Para a implementação, usamos a refência [https://pt.wikipedia.org/wiki/Algoritmo_A*](https://pt.wikipedia.org/wiki/Algoritmo_A*)